In [ ]:
import deeppavlov
import json
#import numpy as np
import tensorflow as tf

#from itertools import chain
#from pathlib import Path

In [ ]:
from deeppavlov.core.commands.train import build_model_from_config
from deeppavlov.core.data.dataset_reader import DatasetReader
#from deeppavlov.core.data.utils import download_decompress
from deeppavlov.core.common.registry import register

In [ ]:
@register('char_lm_dataset_reader')
class CharLmDatasetReader(DatasetReader):
    """
    Dataset reader which reads 
    """
    def read(self, dir_path: str, mode='self_original'):
        dir_path = Path(dir_path)
        dataset = {}
        for dt in ['train', 'valid', 'test']:
            dataset[dt] = self._parse_data(dir_path / '{}_{}.txt'.format(dt, mode))

        return dataset

    @staticmethod
    def _parse_data(filename):
        examples = []
        print(filename)
        curr_persona = []
        curr_dialog_history = []
        persona_done = False
        with filename.open('r') as fin:
            for line in fin:
                line = ' '.join(line.strip().split(' ')[1:])
                your_persona_pref = 'your persona: '
                if line[:len(your_persona_pref)] == your_persona_pref and persona_done:
                    curr_persona = [line[len(your_persona_pref):]]
                    curr_dialog_history = []
                    persona_done = False
                elif line[:len(your_persona_pref)] == your_persona_pref:
                    curr_persona.append(line[len(your_persona_pref):])
                else:
                    persona_done = True
                    x, y, _, candidates = line.split('\t')
                    candidates = candidates.split('|')
                    example = {
                        'persona': curr_persona,
                        'x': x,
                        'y': y,
                        'dialog_history': curr_dialog_history[:],
                        'candidates': candidates,
                        'y_idx': candidates.index(y)
                    }
                    curr_dialog_history.extend([x, y])
                    examples.append(example)

        return examples

In [1]:
print(len('DeepPavlov is an open-source conversational AI library built on TensorFlow and Keras.'))

85


In [4]:
a = 'a' / 'b'
print(a)

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [7]:
def greeting(name: str) -> str:
    return name

print(greeting(1))

1


In [8]:
a, b = (1, 2)

In [9]:
print(a)

1
